# NBA Prediction Modeling

## Luke DiPerna

### August 2023

(pic)

## Project Goal

## Table of Contents

## Data

The source is scraped from basketball-reference. Includes all the stuff...etc. Needs to be aggregated.

## Data Preparation

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3

from sklearn.model_selection import train_test_split, cross_val_predict, cross_val_score, cross_validate, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, classification_report, mean_squared_error

# import tensorflow as tf
# from tensorflow import keras

import statsmodels.api as sm

import warnings
warnings.filterwarnings("ignore")

# set global style
sns.set_theme()

pd.set_option('display.max_columns', None)

seed=99

### sqlite database

In [1]:
conn = sqlite3.connect('NBA-Boxscore-Database.sqlite')

NameError: name 'sqlite3' is not defined

In [ ]:
query = 'SELECT * from game_info'
game_info = pd.read_sql(query, con=conn)
game_info

In [ ]:
query = 'SELECT * from team_stats'
team_stats = pd.read_sql(query, con=conn)
team_stats

In [ ]:
query = 'SELECT * from player_stats'
player_stats = pd.read_sql(query, con=conn)
player_stats

In [ ]:
con.close()

### CSVs

## Modeling

## Results

## Conclusion

## Next Steps